In [41]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
import os
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer

In [36]:
filename = 'ODIR-5K_Training_Annotations(Updated)_V2.xlsx'
df = pd.read_excel(filename)

In [37]:
leftEyeKeywords = df['Left-Diagnostic Keywords'].copy()
rightEyeKeywords = df['Right-Diagnostic Keywords'].copy()

leftEyeKeywords = leftEyeKeywords.str.split("，")
rightEyeKeywords = rightEyeKeywords.str.split("，")

In [39]:
leftEyeKeywords[2]

['laser spot', 'moderate non proliferative retinopathy']

In [43]:
mlb = MultiLabelBinarizer()

In [47]:
resleft = pd.DataFrame(mlb.fit_transform(leftEyeKeywords),
                   columns=mlb.classes_,
                   index=leftEyeKeywords.index)

In [48]:
allDiagnosisLeft = resleft.columns.to_list()
len(allDiagnosisLeft)

82

In [51]:
resright = pd.DataFrame(mlb.fit_transform(rightEyeKeywords),
                   columns=mlb.classes_,
                   index=rightEyeKeywords.index)

allDiagnosisRight = resright.columns.to_list()
len(allDiagnosisRight)

84

In [52]:
allDiagnosis=list(set(allDiagnosisLeft+allDiagnosisRight))
print("total different keys diagnosis :", len(allDiagnosis))

total different keys diagnosis : 105


In [53]:
test_df = df.copy()
doubleDiagnosisRow = []

In [59]:
def getKeyDiagnosisSingle(colName):
  keyDiagnosis = []
  global doubleDiagnosisRow
  store = True
  for row in range(len(test_df[colName])):
    store = True
    if test_df[colName][row] == 1:
      for lable in test_df.columns[7:]:
        if lable == colName:
          continue
        if test_df[lable][row] == 1:
          doubleDiagnosisRow.append(row)
          store = False
          break
        
      if store == True:
        for i in rightEyeKeywords[row]:
          keyDiagnosis.append(i)
        for i in leftEyeKeywords[row]:
          keyDiagnosis.append(i)
      

  keyDiagnosis = list(set(keyDiagnosis))
  print(keyDiagnosis)
  return keyDiagnosis

In [60]:
keyNormal = getKeyDiagnosisSingle(test_df.columns[7])
keyDiabetes = getKeyDiagnosisSingle(test_df.columns[8])
keyGlaucoma = getKeyDiagnosisSingle(test_df.columns[9])
keyCataract = getKeyDiagnosisSingle(test_df.columns[10])
keyAMD = getKeyDiagnosisSingle(test_df.columns[11])
keyHypertension = getKeyDiagnosisSingle(test_df.columns[12])
keyMyopia = getKeyDiagnosisSingle(test_df.columns[13])
keyOtherDisease = getKeyDiagnosisSingle(test_df.columns[14])

['low image quality', 'normal fundus', 'lens dust']
['mild nonproliferative retinopathy', 'proliferative diabetic retinopathy', 'moderate non proliferative retinopathy', 'diabetic retinopathy', 'no fundus image', 'low image quality', 'severe nonproliferative retinopathy', 'normal fundus', 'optic disk photographically invisible', 'severe proliferative diabetic retinopathy']
['glaucoma', 'lens dust', 'normal fundus', 'optic disk photographically invisible', 'suspected glaucoma']
['normal fundus', 'cataract', 'lens dust']
['dry age-related macular degeneration', 'normal fundus', 'wet age-related macular degeneration']
['hypertensive retinopathy']
['myopic maculopathy', 'anterior segment image', 'lens dust', 'myopic retinopathy', 'normal fundus', 'pathological myopia']
['suspected retinal vascular sheathing', 'pigment epithelium proliferation', 'post laser photocoagulation', 'laser spot', 'morning glory syndrome', 'suspected abnormal color of  optic disc', 'optic nerve atrophy', 'lens dust

In [56]:
labelString = ['Normal', 'Diabetes', 'Glaucoma', 'Cataract', 'AMD', 'Hypertension', 'Myopia', 'Abnormalities']
keyAll = [keyNormal, keyDiabetes, keyGlaucoma, keyCataract, keyAMD, keyHypertension, keyMyopia, keyOtherDisease]

In [57]:
for i in range(8):
  print(labelString[i], len(keyAll[i]))

Normal 3
Diabetes 10
Glaucoma 5
Cataract 3
AMD 3
Hypertension 1
Myopia 6
Abnormalities 57


In [58]:
keyNormal

['low image quality', 'normal fundus', 'lens dust']

In [61]:
print("intersect by normal :\n")
for i in range(1,len(keyAll)):
  keyAll[i] = list(set(keyAll[i])-set(keyAll[0]))

intersect by normal :



In [62]:
for i in range(8):
  print(labelString[i], len(keyAll[i]))

Normal 3
Diabetes 8
Glaucoma 3
Cataract 1
AMD 2
Hypertension 1
Myopia 4
Abnormalities 54


In [63]:
print("\nintersect by other :\n")
for i in range(len(keyAll)):
  for j in range(i,len(keyAll)):
    if i == j:
      continue
    else :
      keyAll[i] = list(set(keyAll[i])-set(keyAll[j]))



intersect by other :



In [64]:
for i in range(8):
  print(labelString[i], len(keyAll[i]))

Normal 3
Diabetes 7
Glaucoma 3
Cataract 1
AMD 2
Hypertension 1
Myopia 4
Abnormalities 54


In [66]:
def getAllRecognizedKey(mkeyAll):
  mallkeyDiagnosis = []
  for i in range(len(mkeyAll)):
    mkeyAll[i] = list(set(mkeyAll[i]))
    mallkeyDiagnosis = mallkeyDiagnosis+list(set(mkeyAll[i]))
  return mallkeyDiagnosis

In [67]:
keyNormal, keyDiabetes, keyGlaucoma, keyCataract, keyAMD, keyHypertension, keyMyopia, keyOtherDisease = keyAll[0], keyAll[1], keyAll[2], keyAll[3], keyAll[4], keyAll[5], keyAll[6], keyAll[7]


In [68]:
allkeyDiagnosis = getAllRecognizedKey(keyAll)
len(allkeyDiagnosis)


75

In [69]:
doubleDiagnosisRow = list(set(doubleDiagnosisRow))
print("double lablel row ",len(doubleDiagnosisRow))
doubleDiagnosisRow.sort()

double lablel row  586


In [70]:
notlisted = []
listed = False
for row in doubleDiagnosisRow:
  # print(row)
  for ilist in leftEyeKeywords[row]:
    # print(ilist)
    listed = False
    for j in keyAll:
      if ilist in j:
        listed = True
        break
    if listed == False:
      notlisted.append(ilist)

for row in doubleDiagnosisRow:
  for ilist in rightEyeKeywords[row]:
    listed = False
    for j in keyAll:
      if ilist in j:
        listed = True
        break
    if listed == False:
      notlisted.append(ilist)

-
-
-
-
-
-
-
-
-

In [23]:
img = Image.open("./ODIR-5K_Training_Dataset/0_left.jpg")
img_array = np.array(img)

In [25]:
print(img_array.shape)

(1728, 2304, 3)


In [26]:
df = pd.DataFrame(img_array.reshape(-1, 4), columns=["R", "G", "B", "A"])

In [27]:
df

,R,G,B,A
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
2985979,0,0,0,0
2985980,0,0,0,0
2985981,0,0,0,0
2985982,0,0,0,0
